In [1]:
import random
import csv

# Function to generate a bandwidth matrix for a given number of nodes
def generate_bandwidth_matrix(num_nodes, min_bandwidth=10, max_bandwidth=1000):
    # Create a matrix with zeros or very high values on the diagonal
    bandwidth_matrix = [[0 if i == j else random.randint(min_bandwidth, max_bandwidth)
                         for j in range(num_nodes)] for i in range(num_nodes)]
    return bandwidth_matrix

# Generate bandwidth matrix for 9 nodes
num_nodes = 9
bandwidth_matrix = generate_bandwidth_matrix(num_nodes, min_bandwidth=50, max_bandwidth=1000)  # Bandwidth in Mbps

# Save the bandwidth matrix to a CSV file
with open('bandwidth_matrix.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(bandwidth_matrix)

# Print matrix to verify
for row in bandwidth_matrix:
    print(row)


[0, 599, 80, 775, 80, 799, 462, 771, 216]
[584, 0, 995, 809, 763, 808, 707, 509, 901]
[524, 60, 0, 720, 683, 294, 666, 467, 912]
[620, 268, 741, 0, 890, 966, 532, 445, 167]
[798, 65, 75, 432, 0, 437, 917, 787, 378]
[66, 573, 155, 912, 658, 0, 811, 144, 825]
[249, 660, 674, 941, 951, 837, 0, 625, 245]
[932, 130, 702, 806, 763, 487, 159, 0, 123]
[242, 254, 868, 865, 867, 216, 386, 258, 0]


In [ ]:
'''the code is executed in paralla '''

import random
import csv
import paramiko
from multiprocessing import Pool

# Function to generate a realistic delay matrix
def generate_delay_matrix(num_nodes, base_latency=5, max_additional_latency=50):
    delay_matrix = [[0 for _ in range(num_nodes)] for _ in range(num_nodes)]

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                delay_matrix[i][j] = 0 # node-to-node its self, latency set to zero
            else:
                # additional_latency = random.uniform(0, max_additional_latency)
                additional_latency = random.uniform(0, max_additional_latency)
                distance_factor = abs(i - j) / num_nodes
                simulated_latency = base_latency + additional_latency * distance_factor
                congestion_factor = random.uniform(0.5, 1.5)
                delay_matrix[i][j] = int(simulated_latency * congestion_factor) # change the float to integer
    return delay_matrix

# Generate delay matrix for 9 worker nodes
'''Injecting no latencies'''
# delay_matrix = generate_delay_matrix(num_nodes=9, base_latency=0, max_additional_latency=0)

# Generate delay matrix for 9 worker nodes
delay_matrix = generate_delay_matrix(num_nodes=9, base_latency= 5, max_additional_latency= 50)


# Save the delay matrix to a CSV file
with open('delay_matrix_parallel.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(delay_matrix)

In [ ]:
def apply_bandwidth_between_nodes(source_node_name, username, key_path, interface, bandwidth_matrix, node_details):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.load_system_host_keys()

    try:
        source_node_ip = node_details[source_node_name]['ip']
        client.connect(source_node_ip, username=username, key_filename=key_path)

        # Clear existing rules and set root qdisc
        client.exec_command(f"sudo tc qdisc del dev {interface} root")
        client.exec_command(f"sudo tc qdisc add dev {interface} root handle 1: htb default 1")

        for dst_node, details in node_details.items():
            if dst_node == source_node_name:
                continue

            dst_node_ip = details['ip']
            bandwidth = bandwidth_matrix[source_node_name][dst_node]
            client.exec_command(f"sudo tc class add dev {interface} parent 1: classid 1:1 htb rate {bandwidth}")
            client.exec_command(f"sudo tc filter add dev {interface} protocol ip parent 1:0 prio 1 u32 match ip dst {dst_node_ip} flowid 1:1")

            print(f'Bandwidth set between {source_node_name} and {dst_node}: {bandwidth}')

    except Exception as e:
        print(f"Failed to set bandwidth for {source_node_name}: {e}")
    finally:
        client.close()


In [ ]:


def exclude_src_node(src_node_name, node_details):
    return {name: details for name, details in node_details.items() if name != src_node_name}

def automate_bandwidth_injection(params):
    source_node_name, delay_matrix, node_details = params
    username = node_details[source_node_name]['username']
    key_path = node_details[source_node_name]['key_path']
    interface = 'eth0'  # Assuming the interface name is eth0
    apply_bandwidth_between_nodes(source_node_name, username, key_path, interface, bandwidth_matrix, node_details)

# Assuming correct IP addresses and no duplication in node keys
node_details = {
    'k8s-worker-1': {'ip': '172.26.128.30', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-2': {'ip': '172.26.132.91', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-3': {'ip': '172.26.133.31', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-4': {'ip': '172.26.132.241', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-5': {'ip': '172.26.132.142', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-6': {'ip': '172.26.133.55', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-7': {'ip': '172.26.130.22', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-8': {'ip': '172.26.130.82', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'},
    'k8s-worker-9': {'ip': '172.26.133.118', 'username': 'ubuntu', 'key_path': '/home/ubuntu/.ssh/id_rsa'}
}

# Apply latency injection using the generated delay matrix with multiprocessing
params_list = [(source_node, delay_matrix, node_details) for source_node in node_details.keys()]

if __name__ == '__main__':
    with Pool(processes=len(node_details)) as pool:
        pool.map(automate_bandwidth_injection, params_list)

# plot the delay matrix
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Read the delay matrix from the CSV file
# delay_matrix = pd.read_csv('delay_matrix_parallel.csv', header=None)

# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(delay_matrix, annot=True, fmt=".1f", cmap="viridis", cbar=True)
# plt.title('Latency Heatmap')
# plt.xlabel('Target Node')
# plt.ylabel('Source Node')
# plt.show()
